<a href="https://colab.research.google.com/github/Bigfredoh/Machine-Learning/blob/main/Airline_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
!pip install contractions
import contractions
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')

     |████████████████████████████████| 321 kB 5.2 MB/s 
     |████████████████████████████████| 284 kB 45.9 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85440 sha256=8c4c322749b9b5fd05a619ab2631e141a0e655803f38ecad5ca1dd3033df738a
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [4]:
airline_df = pd.read_csv('Airline-Sentiment.csv', encoding= 'unicode_escape')

In [5]:
airline_df.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,681448150,False,finalized,3,2/25/15 5:24,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2/24/15 11:35,5.703060e+17,NaN,Eastern Time (US & Canada)
1,681448153,False,finalized,3,2/25/15 1:53,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
2,681448156,False,finalized,3,2/25/15 10:01,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2/24/15 11:15,5.703010e+17,Lets Play,Central Time (US & Canada)
3,681448158,False,finalized,3,2/25/15 3:05,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2/24/15 11:15,5.703010e+17,NaN,Pacific Time (US & Canada)
4,681448159,False,finalized,3,2/25/15 5:50,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2/24/15 11:14,5.703010e+17,NaN,Pacific Time (US & Canada)


In [6]:
airline_df.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'airline_sentiment',
       'airline_sentiment:confidence', 'negativereason',
       'negativereason:confidence', 'airline', 'airline_sentiment_gold',
       'name', 'negativereason_gold', 'retweet_count', 'text', 'tweet_coord',
       'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

In [7]:
selected_cols = airline_df[['text', 'airline_sentiment']]
selected_cols.shape

(14640, 2)

In [8]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
from nltk.corpus import stopwords
corpus = []
for i in range(0, 14640):
  text = re.sub(r'[a-z]*[:.]+\S+', '', selected_cols['text'][i])
  text= text.lower()
  text= re.sub('@[A-Za-z0-9_]+', '', text)
  text = re.sub('[^a-zA-Z]', ' ', text)
  text = text.split()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  text = [lmtzr.lemmatize(word) for word in text if not word in set(all_stopwords)]
  text = ' '.join(text)
  corpus.append(contractions.fix(text))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [9]:
corpus

['said',
 'plus added commercial tacky',
 'must mean need take another trip',
 'really aggressive blast obnoxious entertainment guest face amp little recourse',
 'really big bad thing',
 'seriously would pay flight seat playing really bad thing flying va',
 'yes nearly every time fly vx ear worm go away',
 'really missed prime opportunity men without hat parody',
 'well',
 'amazing arrived hour early good',
 'know suicide second leading because death among teen',
 'lt pretty graphic much better minimal iconography',
 'great deal already thinking nd trip amp even gone st trip yet p',
 'flying fabulous seductive sky you take stress away travel',
 'thanks',
 'sfo pdx schedule still mia',
 'excited first cross country flight lax mco heard nothing great thing virgin america daystogo',
 'flew nyc sfo last week fully sit seat due two large gentleman either side help',
 'flying',
 'know would amazingly awesome bos fll please want fly',
 'first fare may three time carrier seat available select'

In [10]:
vectorizer = TfidfVectorizer()
count_matrix = vectorizer.fit_transform(corpus)
count_array = count_matrix.toarray()
X = pd.DataFrame(data=count_array,columns = vectorizer.get_feature_names())
print(X)

        aa  aaaand  aaadvantage  aaalwayslate  ...  zrh  zukes  zurich   zz
0      0.0     0.0          0.0           0.0  ...  0.0    0.0     0.0  0.0
1      0.0     0.0          0.0           0.0  ...  0.0    0.0     0.0  0.0
2      0.0     0.0          0.0           0.0  ...  0.0    0.0     0.0  0.0
3      0.0     0.0          0.0           0.0  ...  0.0    0.0     0.0  0.0
4      0.0     0.0          0.0           0.0  ...  0.0    0.0     0.0  0.0
...    ...     ...          ...           ...  ...  ...    ...     ...  ...
14635  0.0     0.0          0.0           0.0  ...  0.0    0.0     0.0  0.0
14636  0.0     0.0          0.0           0.0  ...  0.0    0.0     0.0  0.0
14637  0.0     0.0          0.0           0.0  ...  0.0    0.0     0.0  0.0
14638  0.0     0.0          0.0           0.0  ...  0.0    0.0     0.0  0.0
14639  0.0     0.0          0.0           0.0  ...  0.0    0.0     0.0  0.0

[14640 rows x 10019 columns]


In [11]:
selected_cols.airline_sentiment.unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [12]:
y = selected_cols.airline_sentiment.map({'neutral':0, 'positive':2, 'negative':1})
y

0        0
1        2
2        0
3        1
4        1
        ..
14635    2
14636    1
14637    0
14638    1
14639    0
Name: airline_sentiment, Length: 14640, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.25, random_state=3)

In [14]:
X_train

,aa,aaaand,aaadvantage,aaalwayslate,aadavantage,aadelay,aadv,aadvantage,aafail,aal,aaron,aateam,ab,aback,abandon,abandoned,abandonment,abassinet,abbreve,abc,abcletjetbluestreamfeed,abcnews,abducted,abi,ability,able,aboard,aboout,abounds,abq,abroad,absolute,absolutely,absorb,absorber,absoulutely,absurd,absurdity,absurdly,abt,...,yout,youth,yow,yponthebeat,ypu,yr,ystday,ystrdy,yuck,yucki,yul,yuma,yummy,yup,yvonne,yvr,yw,ywg,yxe,yxu,yyc,yyj,yyz,yyzua,zabsonre,zagging,zambia,zcc,zero,zfv,zig,zip,zipper,zombie,zone,zoom,zrh,zukes,zurich,zz
12775,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14230,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9160,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9859,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11513,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1688,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [22]:
rfc_pred = rfc.predict(X_test)

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, rfc_pred)
print(cm)
acc_score = accuracy_score(y_test, rfc_pred)*100
print('Accuracy score is:  ' , acc_score)

[[ 359  386   60]
 [ 108 2110   42]
 [  74  198  323]]
Accuracy score is:   76.28415300546449


In [25]:
from sklearn.naive_bayes import GaussianNB
gnb= GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [27]:
gnb_pred= gnb.predict(X_test)

In [28]:
cm = confusion_matrix(y_test, gnb_pred)
print(cm)
acc_score = accuracy_score(y_test,gnb_pred)*100
print('Accuracy score is:  ' , acc_score)

[[ 252  160  393]
 [ 479 1076  705]
 [  88  120  387]]
Accuracy score is:   46.85792349726776
